# 통합 데이터 만드는 과정

In [1]:
import pandas as pd
import pickle

In [2]:
movie_3229 = pd.read_csv('/opt/ml/input/final_project/MBTI_merge_movieLens_3229_movie.csv')
mbti_all = pd.read_csv('/opt/ml/input/final_project/data/Personality_All_168000_rows.csv')
ko_4035 = pd.read_csv('/opt/ml/input/final_project/Content_based/Crawling_ko_information_4035.csv')
data = pd.read_csv('/opt/ml/input/final_project/Crawling/imdb_side_df_EmptyList_to_None.csv') # side information data
data = data.drop_duplicates('Contents',keep='first') # 중복 제거
data = data.drop(['produceers', 'composers'], axis=1) # 수집 하나도 안되서 쓸모 없음


movieId_df = movie_3229[['Contents', 'movieId']]
movieId_df = movieId_df.drop_duplicates('movieId', keep = 'first')

movieId_dict = {}
for i,row in movieId_df.iterrows():
    movieId_dict[row['Contents']] = row['movieId']



merge_df = pd.merge(ko_4035, data, how='left', on='Contents')
merge_df['movieId'] = 0

mnt = 300000.0
for i ,row in merge_df.iterrows():
    if row['Contents'] in list(movieId_dict.keys()):
        merge_df.loc[i,'movieId']= movieId_dict[row['Contents']]
    else:
        merge_df.loc[i,'movieId']= mnt
        mnt+=1


def remove_list_symbol(row):  # DataFrame 만들때 조금 문제가 있었는듯
    try:
        return row['plot'][2:-2]  # 리스트 형태가 아니고 str이다 ex "['Batman ...']" 이런식으로 저장되어있어서 ['와 ']를 제거하는 목적
    except TypeError:
        pass

def remove_list_symbol_2(row):  # DataFrame 만들때 조금 문제가 있었는듯
    try:
        return row['genres'][1:-1]  # 리스트 형태가 아니고 str이다 ex "['Batman ...']" 이런식으로 저장되어있어서 ['와 ']를 제거하는 목적
    except TypeError:
        pass
def remove_list_symbol_3(row):  # DataFrame 만들때 조금 문제가 있었는듯
    try:
        return row['directors'][1:-1]  # 리스트 형태가 아니고 str이다 ex "['Batman ...']" 이런식으로 저장되어있어서 ['와 ']를 제거하는 목적
    except TypeError:
        pass
def remove_list_symbol_4(row):  # DataFrame 만들때 조금 문제가 있었는듯
    try:
        return row['actors'][1:-1]  # 리스트 형태가 아니고 str이다 ex "['Batman ...']" 이런식으로 저장되어있어서 ['와 ']를 제거하는 목적
    except TypeError:
        pass

merge_df['plot'] = merge_df.apply(remove_list_symbol, axis=1)
merge_df['genres'] = merge_df.apply(remove_list_symbol_2, axis=1)
merge_df['directors'] = merge_df.apply(remove_list_symbol_3, axis=1)
merge_df['actors'] = merge_df.apply(remove_list_symbol_4, axis=1)

merge_df

,Contents,ko_title,year,country,ko_genre,ko_plot,rating,popular,img_url,searched_content,genres,plot,directors,actors,movieId
0,Moana (2016),모아나,2016,미국,애니메이션/모험/코미디/가족/판타지/뮤지컬,"바다가 선택한 소녀, 모아나. 저주에 걸린 모투누이 섬을 구하기 위해 머나먼 항해를...",3.8,21만,https://an2-img.amz.wtchn.net/image/v2/3eqoRSv...,Moana,"'Animation', 'Adventure', 'Comedy', 'Family', ...","In Ancient Polynesia, when a terrible curse in...","'Ron Clements', 'John Musker', 'Don Hall', 'Ch...","""Auli'i Cravalho"", 'Dwayne Johnson', 'Rachel H...",73141
1,Inside Out (2015),인사이드 아웃,2015,미국,코미디/가족/드라마/판타지/애니메이션/모험,모든 사람의 머릿속에 존재하는 감정 컨트롤 본부. 그곳에서 불철주야 열심히 일하는 ...,4.1,79만,https://an2-img.amz.wtchn.net/image/v2/o-maJ99...,Inside Out,"'Animation', 'Adventure', 'Comedy', 'Drama', '...",After young Riley is uprooted from her Midwest...,"'Pete Docter', 'Ronnie Del Carmen'","'Amy Poehler', 'Phyllis Smith', 'Richard Kind'...",122470
2,Zootopia (2016),주토피아,2016,미국,모험/애니메이션/액션/코미디/가족/키즈,교양 있고 세련된 라이프 스타일을 주도하는 도시 주토피아. 이 곳을 단숨에 혼란에 ...,4.2,92만,no image,Zootopia,"'Animation', 'Adventure', 'Comedy', 'Crime', '...","In a city of anthropomorphic animals, a rookie...","'Byron Howard', 'Rich Moore', 'Jared Bush'","'Ginnifer Goodwin', 'Jason Bateman', 'Idris El...",152081
3,Legally Blonde (2001),금발이 너무해,2001,미국,로맨스/코미디/로맨틱 코미디/드라마,부유한 집안 출신에 금발과 제법 큰 가슴을 지닌 여학생 클럽 회장 엘 우즈. 세상 ...,3.6,30만,https://an2-img.amz.wtchn.net/image/v2/xSEXNF5...,Legally Blonde,"'Comedy', 'Romance'",None,'Robert Luketic',"'Reese Witherspoon', 'Luke Wilson', 'Selma Bla...",4447
4,Dead Poets Society (1989),죽은 시인의 사회,1989,미국,드라마,"“오 캡틴, 나의 캡틴”",4.2,63만,https://an2-img.amz.wtchn.net/image/v2/Z9BhW72...,Dead Poets Society,"'Comedy', 'Drama'",Maverick teacher John Keating uses poetry to e...,'Peter Weir',"'Robin Williams', 'Robert Sean Leonard', 'Etha...",1246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4030,Cloak & Dagger (2018),클록 & 대거 시즌 1,2018,미국,TV드라마/액션/드라마/모험,탠디와 타이론 두 명의 10대가 사랑을 키워가다가 신비한 초능력을 얻는다. 탠디는 ...,2.9,151,https://an2-img.amz.wtchn.net/image/v2/65G5eC6...,Cloak & Dagger,"'Action', 'Adventure', 'Drama', 'Sci-Fi'",Two teenagers from very different backgrounds ...,None,"'Olivia Holt', 'Aubrey Joseph', 'Gloria Reuben...",301017
4031,Peacemaker (2022),피스메이커,2022,미국,액션/모험/코미디/TV드라마,'수어사이드 스쿼드'의 사건 이후를 배경으로 하는 드라마 시리즈,3.9,"1,619",https://an2-img.amz.wtchn.net/image/v2/anD7AbB...,Peacemaker,"'Action', 'Adventure', 'Comedy', 'Crime', 'Fan...",Picking up where The Suicide Squad (2021) left...,None,"'John Cena', 'Danielle Brooks', 'Freddie Strom...",301018
4032,Guardians of the Galaxy (2015),Guardians of the Galaxy,no_data,no_data,no_data,no_data,no_data,no_data,no data,Guardians of the Galaxy,"'Action', 'Adventure', 'Comedy', 'Sci-Fi'",A group of intergalactic criminals must pull t...,'James Gunn',"'Chris Pratt', 'Zoe Saldana', 'Dave Bautista',...",301019
4033,Icon Comics,Icon Comics,no_data,no_data,no_data,no_data,no_data,no_data,no data,Iron Sky: The Coming Race,"'Action', 'Adventure', 'Comedy', 'Sci-Fi'",A follow-up to the film Iron Sky (2012) in whi...,'Timo Vuorensola',"'Lara Rossi', 'Vladimir Burlakov', 'Kit Dale',...",301020


이렇게 만들어진 merge_df를 "content_based_4035.csv"로 저장하였다  
(혼란을 막기 위해 같은 이름으로 파일을 덮어씌우는 행위는 지양하기 바랍니다)

In [6]:
merge_df.nunique()

Contents            4035
ko_title            3919
year                 100
country              479
ko_genre            1287
ko_plot             3344
rating                41
popular             1554
img_url             3636
searched_content    3925
genres               952
plot                2960
directors           2218
actors              3945
movieId             4035
dtype: int64

----------------

# CharacterId 만드는 과정

In [ ]:
all_char = pd.read_csv('/opt/ml/input/final_project/data/Personality_All_168000_rows.csv')

MCU = ['MCU: The Villains', 'MCU: The Heroes', 'MCU: Everyone Else', ]
DC = ['DC Extended Universe', 'DC Comics', 'DC Super Hero Girls', 'DC Animated Movie Universe']

all_char.loc[all_char['Contents'].isin(MCU), 'Contents'] = 'Avengers'
all_char.loc[all_char['Contents'].isin(DC), 'Contents'] = 'Justice League'

cbf = pd.read_csv('/opt/ml/input/final_project/Content_based/content_based_4035.csv')

char_df = all_char.drop(['Page', 'category'], axis=1)
char_df['CharacterId'] = 0

merge_df = pd.merge(char_df, cbf, how='left', on='Contents')

character_df = merge_df.loc[merge_df['movieId'].isna() != True]

character_df = character_df.drop(['ko_title', 'year', 'country', 'ko_genre', 'ko_plot', 'rating', 'popular', 'searched_content', 'genres', 'plot',
       'directors', 'actors','img_url'],axis=1)

cnt = 0
for i, row in character_df.iterrows():
    character_df.loc[i,'CharacterId']= cnt
    cnt+=1